<a href="https://colab.research.google.com/github/Schmutzz/PV_Forecast_Platone/blob/master/platone_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import pandas as pd
import sqlite3
import timeit

# establish sql connection
db_path = "data/input_data.db"
conn = sqlite3.connect(db_path)
